<a href="https://colab.research.google.com/github/Vicks-2019/Machine-Leaning/blob/main/Scikit_learn_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
from sklearn.compose import ColumnTransformer 
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV

from sklearn.model_selection import train_test_split

from pandas_profiling import ProfileReport

import numpy as np

import warnings
warnings.filterwarnings('ignore')

In [3]:
from google.colab import files
uploaded = files.upload()

Saving customer_churn.csv to customer_churn.csv


In [5]:
df = pd.read_csv('customer_churn.csv')
df

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,Yes,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,No,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,6840-RESVB,Male,0,Yes,Yes,24,Yes,Yes,DSL,Yes,No,Yes,Yes,Yes,Yes,One year,Yes,Mailed check,84.80,1990.5,No
7039,2234-XADUH,Female,0,Yes,Yes,72,Yes,Yes,Fiber optic,No,Yes,Yes,No,Yes,Yes,One year,Yes,Credit card (automatic),103.20,7362.9,No
7040,4801-JZAZL,Female,0,Yes,Yes,11,No,No phone service,DSL,Yes,No,No,No,No,No,Month-to-month,Yes,Electronic check,29.60,346.45,No
7041,8361-LTMKD,Male,1,Yes,No,4,Yes,Yes,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Mailed check,74.40,306.6,Yes


In [7]:
df.describe()

,SeniorCitizen,tenure,MonthlyCharges
count,7043.000000,7043.000000,7043.000000
mean,0.162147,32.371149,64.761692
std,0.368612,24.559481,30.090047
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.500000
50%,0.000000,29.000000,70.350000
75%,0.000000,55.000000,89.850000
max,1.000000,72.000000,118.750000


In [8]:
X = df.drop(columns=['Churn'])
y = df['Churn']

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=124)

In [10]:

drop_feat=  ['customerID','gender','PhoneService','MultipleLines', 'PaperlessBilling','PaymentMethod']
numeric_features = ['tenure', 'TotalCharges']
categorical_features = ['SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'InternetService','OnlineSecurity','OnlineBackup','DeviceProtection','TechSupport','StreamingTV','StreamingMovies','Contract']

In [11]:
drop_transformer = ColumnTransformer(transformers=[('drop_columns', 'drop', drop_feat)], remainder='passthrough')

In [12]:

pipeline = Pipeline([('drop_column', drop_transformer)])

In [13]:

pipeline.fit(X_train)

Pipeline(memory=None,
         steps=[('drop_column',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('drop_columns', 'drop',
                                                  ['customerID', 'gender',
                                                   'PhoneService',
                                                   'MultipleLines',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])],
                                   verbose=False))],
         verbose=False)

In [14]:
transformed_train=pipeline.transform(X_train)

In [15]:
transformed_train.view()

array([[0, 'No', 'Yes', ..., 'Month-to-month', 19.3, '19.3'],
       [0, 'Yes', 'No', ..., 'Month-to-month', 55.55, '1405.3'],
       [0, 'Yes', 'Yes', ..., 'Two year', 19.55, '1463.45'],
       ...,
       [0, 'No', 'No', ..., 'Month-to-month', 74.35, '533.6'],
       [0, 'No', 'No', ..., 'Month-to-month', 45.15, '98.5'],
       [1, 'No', 'No', ..., 'Month-to-month', 85.25, '2287.25']],
      dtype=object)

In [16]:
df.isna().sum()

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64

In [17]:
df.replace(r'^\s*$', np.nan, regex=True).isna().sum()

customerID           0
gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
StreamingTV          0
StreamingMovies      0
Contract             0
PaperlessBilling     0
PaymentMethod        0
MonthlyCharges       0
TotalCharges        11
Churn                0
dtype: int64

In [18]:
def remove_spaces(input_df):
    input_df["TotalCharges"] = input_df["TotalCharges"].replace(r'^\s*$', np.nan, regex=True)
    return input_df

In [19]:
class SpaceImputeTransformer():
    def __init__(self, func):
        self.func = func

    def transform(self, input_df, **transform_params):
        return self.func(input_df)

    def fit(self, X, y=None, **fit_params):
        return self

In [21]:
pipeline = Pipeline([
                     ('space_remover', SpaceImputeTransformer(remove_spaces)),
                     ('drop_column', drop_transformer)

                    ])

In [22]:
pipeline.fit(X_train)

Pipeline(memory=None,
         steps=[('space_remover',
                 <__main__.SpaceImputeTransformer object at 0x7fc2305f2278>),
                ('drop_column',
                 ColumnTransformer(n_jobs=None, remainder='passthrough',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('drop_columns', 'drop',
                                                  ['customerID', 'gender',
                                                   'PhoneService',
                                                   'MultipleLines',
                                                   'PaperlessBilling',
                                                   'PaymentMethod'])],
                                   verbose=False))],
         verbose=False)

In [23]:

transformed_train=pipeline.transform(X_train)

In [24]:
transformed_train

array([[0, 'No', 'Yes', ..., 'Month-to-month', 19.3, '19.3'],
       [0, 'Yes', 'No', ..., 'Month-to-month', 55.55, '1405.3'],
       [0, 'Yes', 'Yes', ..., 'Two year', 19.55, '1463.45'],
       ...,
       [0, 'No', 'No', ..., 'Month-to-month', 74.35, '533.6'],
       [0, 'No', 'No', ..., 'Month-to-month', 45.15, '98.5'],
       [1, 'No', 'No', ..., 'Month-to-month', 85.25, '2287.25']],
      dtype=object)

In [25]:
np.isnan(transformed_train[:,-1].astype(np.float)).sum()

10

In [26]:
np.isnan(pipeline.transform(X_test)[:,-1].astype(np.float)).sum()

1

In [27]:
numeric_transformer = Pipeline(steps=[
                                     ('meanimputer', SimpleImputer(strategy='mean')),
                                     ('stdscaler', StandardScaler())
                                     ])

categorical_transformer = Pipeline(steps=[
                                         ('onehotenc', OneHotEncoder(handle_unknown='ignore'))
                                         ])

In [28]:
col_transformer = ColumnTransformer(transformers=[('drop_columns', 'drop', drop_feat),
                                                   ('numeric_processing',numeric_transformer, numeric_features),
                                                    ('categorical_processing', categorical_transformer, categorical_features)
                                                  ], remainder='drop')

In [29]:
pipeline = Pipeline([
                     ('space_remover', SpaceImputeTransformer(remove_spaces)),
                     ('transform_column', col_transformer),
                    ])


In [30]:
pipeline.fit(X_train)

Pipeline(memory=None,
         steps=[('space_remover',
                 <__main__.SpaceImputeTransformer object at 0x7fc2305cec18>),
                ('transform_column',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('drop_columns', 'drop',
                                                  ['customerID', 'gender',
                                                   'PhoneService',
                                                   'MultipleLines',
                                                   'PaperlessBilling',
                                                   'PaymentMethod']),
                                                 ('numer...
                                                           steps=[('onehotenc',
                                                                   OneHotEncoder

In [31]:
transformed_train=pipeline.transform(X_train)

In [32]:
transformed_train.shape

(4930, 34)

In [33]:
pd.DataFrame(transformed_train)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33
0,-1.287226,-1.008875,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.311327,-0.392971,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.599807,-0.367131,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.351990,-0.602716,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,0.583246,1.010232,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4925,-1.002589,-0.772179,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4926,0.420596,-0.489445,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4927,-1.043251,-0.780333,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4928,-1.246563,-0.973681,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [34]:
pipeline.transform(X_test)

array([[ 1.55914459,  2.40019296,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [-1.16523858, -0.88273919,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       [-0.35198988, -0.41821168,  1.        , ...,  1.        ,
         0.        ,  0.        ],
       ...,
       [ 0.5425837 , -0.60547135,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [ 1.55914459,  2.44105325,  1.        , ...,  0.        ,
         0.        ,  1.        ],
       [-0.10801526, -0.73969485,  1.        , ...,  0.        ,
         1.        ,  0.        ]])

In [35]:
pipeline.named_steps['transform_column'].transformers_[2][1]

Pipeline(memory=None,
         steps=[('onehotenc',
                 OneHotEncoder(categories='auto', drop=None,
                               dtype=<class 'numpy.float64'>,
                               handle_unknown='ignore', sparse=True))],
         verbose=False)

In [36]:

cat_cols=pipeline.named_steps['transform_column'].transformers_[2][1].named_steps['onehotenc'].get_feature_names(categorical_features)

In [37]:
numeric_features+cat_cols.tolist()

['tenure',
 'TotalCharges',
 'SeniorCitizen_0',
 'SeniorCitizen_1',
 'Partner_No',
 'Partner_Yes',
 'Dependents_No',
 'Dependents_Yes',
 'PhoneService_No',
 'PhoneService_Yes',
 'InternetService_DSL',
 'InternetService_Fiber optic',
 'InternetService_No',
 'OnlineSecurity_No',
 'OnlineSecurity_No internet service',
 'OnlineSecurity_Yes',
 'OnlineBackup_No',
 'OnlineBackup_No internet service',
 'OnlineBackup_Yes',
 'DeviceProtection_No',
 'DeviceProtection_No internet service',
 'DeviceProtection_Yes',
 'TechSupport_No',
 'TechSupport_No internet service',
 'TechSupport_Yes',
 'StreamingTV_No',
 'StreamingTV_No internet service',
 'StreamingTV_Yes',
 'StreamingMovies_No',
 'StreamingMovies_No internet service',
 'StreamingMovies_Yes',
 'Contract_Month-to-month',
 'Contract_One year',
 'Contract_Two year']

In [38]:

for name, estimator, features in pipeline.named_steps['transform_column'].transformers_:
  print(name)
  print(features)

drop_columns
['customerID', 'gender', 'PhoneService', 'MultipleLines', 'PaperlessBilling', 'PaymentMethod']
numeric_processing
['tenure', 'TotalCharges']
categorical_processing
['SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract']
remainder
[18]


In [39]:
pd.DataFrame(transformed_train, columns=numeric_features+cat_cols.tolist())

,tenure,TotalCharges,SeniorCitizen_0,SeniorCitizen_1,Partner_No,Partner_Yes,Dependents_No,Dependents_Yes,PhoneService_No,PhoneService_Yes,InternetService_DSL,InternetService_Fiber optic,InternetService_No,OnlineSecurity_No,OnlineSecurity_No internet service,OnlineSecurity_Yes,OnlineBackup_No,OnlineBackup_No internet service,OnlineBackup_Yes,DeviceProtection_No,DeviceProtection_No internet service,DeviceProtection_Yes,TechSupport_No,TechSupport_No internet service,TechSupport_Yes,StreamingTV_No,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_Month-to-month,Contract_One year,Contract_Two year
0,-1.287226,-1.008875,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0
1,-0.311327,-0.392971,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
2,1.599807,-0.367131,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
3,-0.351990,-0.602716,0.0,1.0,0.0,1.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0
4,0.583246,1.010232,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4925,-1.002589,-0.772179,0.0,1.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0
4926,0.420596,-0.489445,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0
4927,-1.043251,-0.780333,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
4928,-1.246563,-0.973681,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0


In [40]:

pipeline = Pipeline([
                     ('space_remover', SpaceImputeTransformer(remove_spaces)),
                     ('transform_column', col_transformer),
                     ('logistics', LogisticRegression())
                    ])

In [41]:
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
         steps=[('space_remover',
                 <__main__.SpaceImputeTransformer object at 0x7fc230570eb8>),
                ('transform_column',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('drop_columns', 'drop',
                                                  ['customerID', 'gender',
                                                   'PhoneService',
                                                   'MultipleLines',
                                                   'PaperlessBilling',
                                                   'PaymentMethod']),
                                                 ('numer...
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
        

In [42]:
pipeline.score(X_test, y_test)

0.8078561287269286

In [43]:

y_pred = pipeline.predict(X_test)

In [44]:
from sklearn.metrics import confusion_matrix

In [45]:
confusion_matrix(y_test, y_pred)

array([[1409,  144],
       [ 262,  298]])

In [46]:
import joblib

In [47]:
joblib.dump(pipeline, 'pipeline.pkl')

['pipeline.pkl']

In [48]:
pipeline_loaded = joblib.load('pipeline.pkl')

In [51]:
pipeline_loaded.score(X_test, y_test)

0.8078561287269286

In [52]:

pipeline_loaded

Pipeline(memory=None,
         steps=[('space_remover',
                 <__main__.SpaceImputeTransformer object at 0x7fc230570400>),
                ('transform_column',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('drop_columns', 'drop',
                                                  ['customerID', 'gender',
                                                   'PhoneService',
                                                   'MultipleLines',
                                                   'PaperlessBilling',
                                                   'PaymentMethod']),
                                                 ('numer...
                                                   'DeviceProtection',
                                                   'TechSupport', 'StreamingTV',
        

In [53]:

pipeline_lr = Pipeline([
                     ('space_remover', SpaceImputeTransformer(remove_spaces)),
                     ('transform_column', col_transformer),
                     ('logistics', LogisticRegression())
                    ])

In [54]:

col_transformer_rf = ColumnTransformer(transformers=[('drop_columns', 'drop', drop_feat),
                                                   ('numeric_processing',numeric_transformer, numeric_features)                                                 
                                                  ], remainder='drop')

In [55]:
pipeline_rf = Pipeline([
                     ('space_remover', SpaceImputeTransformer(remove_spaces)),
                     ('transform_column', col_transformer_rf),
                     ('RForest', RandomForestClassifier())
                    ])

In [56]:
pipeline_all=[pipeline_lr, pipeline_rf]

In [57]:
for pipe in pipeline_all:
  pipe.fit(X_train, y_train)

In [58]:
for i,model in enumerate(pipeline_all):
  print(model.score(X_test, y_test))

0.8078561287269286
0.7373402744912447


In [59]:
grid_param = [
{"RForest": [RandomForestClassifier()],
"RForest__n_estimators":[10,50,100],  
"RForest__max_leaf_nodes": [5, 6,8]}]
gridsearch = GridSearchCV(pipeline_rf, grid_param, cv=5, verbose=1,n_jobs=-1)

In [60]:
best_model = gridsearch.fit(X_train,y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 2 concurrent workers.
[Parallel(n_jobs=-1)]: Done  45 out of  45 | elapsed:    8.9s finished


In [61]:
best_model.score(X_test,y_test)

0.7624230951254141